In [3]:
import requests
import pandas as pd

req = requests.get("https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-donnees-statiques/records?")
wb = req.json()

url_belibstat = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-donnees-statiques/records?"

url_count = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptages-routiers-permanents/records?"

url_belibdyn = "https://parisdata.opendatasoft.com/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-disponibilite-temps-reel/records?"

In [21]:
print(req.content[:1000])

b'{"total_count": 1947, "results": [{"id_pdc_local": "FR*V75*E9015*06*2", "statut_pdc": "En service", "arrondissement": "15e Arrondissement", "id_station_local": "FR*V75*P9015*06", "id_station_itinerance": "FRV75P901506", "nom_station": "Paris | Rue Desnouettes 55", "code_insee_commune": "75115", "implantation_station": "Voirie", "nbre_pdc": "3", "date_maj": "2024-07-01", "condition_acces": "Acc\\u00e8s libre", "adresse_station": "55 Rue Desnouettes 75015 Paris", "coordonneesxy": {"lon": 2.286912, "lat": 48.83571}, "gratuit": "false", "paiement_acte": "true", "paiement_cb": "false", "paiement_autre": "true", "reservation": "false", "observations": ["Paiement autre - badge RFID Belib ou autre, application Belib, QRCode sur chaque PDC, site internet Belib", "Plus d\'information - https://belib.paris.fr"], "num_pdl": "07436179348549", "siren_amenageur": "531680445", "contact_amenageur": "assistance.technique@tevgo.fr", "nom_amenageur": "TOTALENERGIES", "nom_operateur": "TOTALENERGIES", "c

In [4]:
import pandas as pd



def get_dpe_from_url(url):

    req = requests.get(url)
    wb = req.json()
    df = pd.json_normalize(wb["results"])

    

    return df


In [5]:
dpe_belibstat = get_dpe_from_url(url_belibstat)
dpe_count = get_dpe_from_url(url_count)
dpe_belibdyn = get_dpe_from_url(url_belibdyn)
dpe_belibstat.head(2)



,id_pdc_local,statut_pdc,arrondissement,id_station_local,id_station_itinerance,nom_station,code_insee_commune,implantation_station,nbre_pdc,date_maj,...,prise_type_ef,prise_type_2,prise_type_combo_ccs,prise_type_chademo,prise_type_autre,prise_type_3,horaires,raccordement,coordonneesxy.lon,coordonneesxy.lat
0,FR*V75*E9015*06*2,En service,15e Arrondissement,FR*V75*P9015*06,FRV75P901506,Paris | Rue Desnouettes 55,75115,Voirie,3,2024-07-01,...,True,False,False,False,False,False,24/7,Direct,2.286912,48.835710
1,FR*V75*E9015*07*3,En service,15e Arrondissement,FR*V75*P9015*07,FRV75P901507,Paris | Rue Theophraste Renaudot 21,75115,Voirie,3,2024-07-01,...,True,True,True,True,False,True,24/7,Direct,2.296744,48.841286


In [27]:
dpe_count.head(2)

,iu_ac,libelle,t_1h,q,k,etat_trafic,iu_nd_amont,libelle_nd_amont,iu_nd_aval,libelle_nd_aval,etat_barre,date_debut,date_fin,geo_point_2d.lon,geo_point_2d.lat,geo_shape.type,geo_shape.geometry.coordinates,geo_shape.geometry.type
0,6529,Pyrenees,2023-11-06T02:00:00+00:00,49.0,NaN,Inconnu,3384,Pyrenees-La_Mare,3377,Belleville-Pyrenees,Invalide,2003-03-11,2023-01-01,2.386763,48.873903,Feature,"[[2.388005680304629, 48.87384643458903], [2.38...",LineString
1,6529,Pyrenees,2023-11-06T01:00:00+00:00,76.0,NaN,Inconnu,3384,Pyrenees-La_Mare,3377,Belleville-Pyrenees,Invalide,2003-03-11,2023-01-01,2.386763,48.873903,Feature,"[[2.388005680304629, 48.87384643458903], [2.38...",LineString


In [20]:
dpe_belibdyn.head(2)

,id_pdc,statut_pdc,url_description_pdc,last_updated,adresse_station,code_insee_commune,arrondissement,coordonneesxy.lon,coordonneesxy.lat,coordonneesxy
0,FR*V75*EPX11*05*4,Disponible,https://parisdata.opendatasoft.com/explore/dat...,2022-06-13T02:30:01+00:00,8 Boulevard Voltaire 75011 Paris,75111,11e Arrondissement,2.366438,48.865765,NaN
1,FR*V75*EPX13*05*1,Inconnu,https://parisdata.opendatasoft.com/explore/dat...,2023-06-06T02:30:04+00:00,188 Avenue de Choisy 75013 Paris,75113,13e Arrondissement,2.356917,48.830390,NaN


# Affichage carte 

In [6]:
import geopandas as gpd

def get_geo_from_dpe(df):

    # Supprimer les lignes où les coordonnées sont manquantes
    df = df.dropna(subset=["coordonneesxy.lon", "coordonneesxy.lat"])

    # Créer un GeoDataFrame avec des points géographiques
    dpe = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["coordonneesxy.lon"], df["coordonneesxy.lat"]),crs=4326
          
    )

    return dpe

# Exemple d'appel de la fonction
geo_belibdyn = get_geo_from_dpe(dpe_belibdyn)


geo_belibdyn.head(2)

In [7]:
geo_belibdyn.head(2)

,id_pdc,statut_pdc,url_description_pdc,last_updated,adresse_station,code_insee_commune,arrondissement,coordonneesxy.lon,coordonneesxy.lat,coordonneesxy,geometry
0,FR*V75*EPX11*05*4,Disponible,https://parisdata.opendatasoft.com/explore/dat...,2022-06-13T02:30:01+00:00,8 Boulevard Voltaire 75011 Paris,75111,11e Arrondissement,2.366438,48.865765,NaN,POINT (2.36644 48.86576)
1,FR*V75*EPX13*05*1,Inconnu,https://parisdata.opendatasoft.com/explore/dat...,2023-06-06T02:30:04+00:00,188 Avenue de Choisy 75013 Paris,75113,13e Arrondissement,2.356917,48.830390,NaN,POINT (2.35692 48.83039)


In [10]:
import seaborn as sns
import folium
import pandas as pd
from matplotlib.colors import to_hex

# Palette de couleurs
palette = sns.color_palette("tab10", 10)  # 10 couleurs distinctes

def interactive_map_dpe(dpe):
    # Afficher les lignes contenant des NaN dans les coordonnées avant nettoyage
    print("Lignes avec NaN dans les coordonnées avant nettoyage:")
    print(dpe[dpe[["coordonneesxy.lat", "coordonneesxy.lon"]].isna().any(axis=1)])

    # Convertir explicitement les coordonnées en numériques (si nécessaire)
    dpe["coordonneesxy.lat"] = pd.to_numeric(dpe["coordonneesxy.lat"], errors="coerce")
    dpe["coordonneesxy.lon"] = pd.to_numeric(dpe["coordonneesxy.lon"], errors="coerce")

    # Supprimer les lignes avec des NaN dans coordonneesxy.lat ou coordonneesxy.lon
    dpe_clean = dpe.dropna(subset=["coordonneesxy.lat", "coordonneesxy.lon"])


    # Associer une couleur unique à chaque statut
    unique_statuses = dpe_clean["statut_pdc"].unique()
    status_colors = {status: to_hex(palette[i % len(palette)]) for i, status in enumerate(unique_statuses)}

    # Ajouter la colonne de couleur basée sur statut_pdc
    dpe_clean["color"] = dpe_clean["statut_pdc"].map(status_colors)

    # Calculer le centre de la carte et ses limites
    center = dpe_clean[["coordonneesxy.lat", "coordonneesxy.lon"]].mean().values.tolist()
    sw = dpe_clean[["coordonneesxy.lat", "coordonneesxy.lon"]].min().values.tolist()
    ne = dpe_clean[["coordonneesxy.lat", "coordonneesxy.lon"]].max().values.tolist()

    print(f"Centre de la carte : {center}")  # Afficher le centre pour vérifier

    # Initialiser la carte Folium
    m = folium.Map(location=center, tiles="OpenStreetMap")

    # Ajouter les marqueurs à la carte
    for i in range(len(dpe_clean)):
        folium.Marker(
            [dpe_clean.iloc[i]["coordonneesxy.lat"], dpe_clean.iloc[i]["coordonneesxy.lon"]],
            popup=f"Adresse: {dpe_clean.iloc[i]['adresse_station']}, <br>"
                  f"Arrondissement: {dpe_clean.iloc[i]['arrondissement']}, <br>"
                  f"Statut: {dpe_clean.iloc[i]['statut_pdc']}",
            icon=folium.Icon(
                color="black", icon="info-sign", icon_color=dpe_clean.iloc[i]["color"]
            ),
        ).add_to(m)

    # Ajuster les limites de la carte
    m.fit_bounds([sw, ne])

    return m

# Appel de la fonction avec vos données
m = interactive_map_dpe(geo_belibdyn)

# Si m est None, cela signifie qu'il y a des NaN persistants dans les coordonnées


Lignes avec NaN dans les coordonnées avant nettoyage:
Empty GeoDataFrame
Columns: [id_pdc, statut_pdc, url_description_pdc, last_updated, adresse_station, code_insee_commune, arrondissement, coordonneesxy.lon, coordonneesxy.lat, coordonneesxy, geometry]
Index: []
Centre de la carte : [48.8635501111111, 2.3508212222222222]


In [11]:
m